In [1]:
import pymc as pm
import numpy as np

n_samples = 100
n_features = 10
X = np.zeros((n_samples, n_features))
y = np.ones((n_samples, 1))

with pm.Model() as logistic_model:
    
    beta_prior = pm.MvNormal('beta_prior', mu=np.zeros(n_features), cov=np.eye(n_features), shape=n_features)
    linear_combination = pm.math.dot(X, beta_prior)
    p_logistic = pm.math.sigmoid(linear_combination)
    observations = pm.Bernoulli('observations', p=p_logistic, observed=y)

with logistic_model:
    inference_data = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_prior]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 4 seconds.


In [ ]:
import pymc as pm
import numpy as np
import arviz as az

np.random.seed(365)

n_samples = 100
n_features = 1
true_beta = 2.5
noise_std_dev = 1

X = np.random.normal(size=(n_samples, n_features))
mu = X * true_beta
y = np.random.normal(mu.flatten(), scale=noise_std_dev)

with pm.Model() as robust_regression_model:
    beta = pm.Normal('beta', mu=0, sd=10)
    nu = pm.Exponential('nu', 1/29) + 1
    lambda_prior = pm.Gamma('lambda_prior', alpha=nu/2, beta=nu/2, shape=n_samples)
    sigma_prior = pm.Deterministic('sigma_prior', 1 / np.sqrt(lambda_prior))
    y_observed = pm.Normal('y_observed', mu=X[:, 0] * beta, sd=sigma_prior, observed=y)
    trace = pm.sample(1000, target_accept=0.9, return_inferencedata=True)

lambda_posterior_mean = az.summary(trace, var_names=['lambda_prior'])['mean'].values
outlier_cutoff = np.quantile(lambda_posterior_mean, 0.05)
potential_outliers = np.where(lambda_posterior_mean < outlier_cutoff)[0]

print(f"Identified potential outliers: {potential_outliers}")